In [ ]:
# install octave
!sudo apt-get -qq update
!sudo apt-get -qq install octave octave-signal liboctave-dev

# # install oct2py that compatible with colab
import os

from pkg_resources import get_distribution

os.system(f"pip install -qq"
          f" ipykernel=={get_distribution('ipykernel').version}"
          f" ipython=={get_distribution('ipython').version}"
          f" tornado=={get_distribution('tornado').version}"
          f" oct2py"
          )

# install packages
!pip install -qq matpower matpowercaseframes

In [ ]:
import oct2py

import matpower

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.6.0
matpower version: 7.1.0.2.1.4


In [ ]:
import os

import numpy as np
import pandas as pd
from matpowercaseframes import CaseFrames
from matpowercaseframes.constants import COLUMNS

from matpower import path_matpower, start_instance

m = start_instance()

## Inspect data

In [ ]:
path = os.path.join(path_matpower, 'data/case9.m')

In [ ]:
cf = CaseFrames(path)

In [ ]:
sol = m.loadcase('case9', verbose=False)
pd.DataFrame(sol['gen'], columns=cf.gen.columns)

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
0,1.0,72.3,27.03,300.0,-300.0,1.040,100.0,1.0,250.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,163.0,6.54,300.0,-300.0,1.025,100.0,1.0,300.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,85.0,-10.95,300.0,-300.0,1.025,100.0,1.0,270.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Run base

In [ ]:
mpc = m.runpf(sol, verbose=True)


MATPOWER Version 8.0b1, 22-Dec-2022 -- AC Power Flow (Newton)

Newton's method power flow (power balance, polar) converged in 4 iterations.

Converged in 0.24 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transform

## Copy and editing gen

In [ ]:
new_gen = sol['gen'][0].copy()  # copy first generator, must use .copy()
new_gen_cost = sol['gencost'][0].copy()

new_gen[COLUMNS['gen'].index('GEN_BUS')] = 7  # connect to bus 8

sol['gen'] = np.vstack((sol['gen'], new_gen))
sol['gencost'] = np.vstack((sol['gencost'], new_gen_cost))

pd.DataFrame(sol['gen'], columns=cf.gen.columns)

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
0,1.0,72.3,27.03,300.0,-300.0,1.040,100.0,1.0,250.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,163.0,6.54,300.0,-300.0,1.025,100.0,1.0,300.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,85.0,-10.95,300.0,-300.0,1.025,100.0,1.0,270.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7.0,72.3,27.03,300.0,-300.0,1.040,100.0,1.0,250.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
mpc = m.runpf(sol, verbose=True)


MATPOWER Version 8.0b1, 22-Dec-2022 -- AC Power Flow (Newton)

Newton's method power flow (power balance, polar) converged in 4 iterations.

Converged in 0.12 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity    1070.0       -1200.0 to 1200.0
Generators         4     On-line Capacity      1070.0       -1200.0 to 1200.0
Committed Gens     4     Generation (actual)    323.3              35.0
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         8.34             61.41
Transfo

In [ ]:
pd.DataFrame(mpc['gen'], columns=COLUMNS['gen'][:mpc['gen'].shape[1]])

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
0,1.0,3.035265,34.443294,300.0,-300.0,1.040,100.0,1.0,250.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,163.000000,-5.229211,300.0,-300.0,1.025,100.0,1.0,300.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,85.000000,-21.204423,300.0,-300.0,1.025,100.0,1.0,270.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7.0,72.300000,27.030000,300.0,-300.0,1.040,100.0,1.0,250.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
